# Minimal Example pandapipes

## Creating a low pressure gas network

We consider the following simple 3-junction example network with an ideal valve as a minimal example:

<img src="pics/minimal_example.png">

The above network can be created in pandapipes as follows:

In [1]:
import pandapipes as pp

#create empty net
net = pp.create_empty_network(fluid="lgas")

# create junction
j1 = pp.create_junction(net, pn_bar=1.05, tfluid_k=293.15, name="Junction 1")
j2 = pp.create_junction(net, pn_bar=1.05, tfluid_k=293.15, name="Junction 2")    
j3 = pp.create_junction(net, pn_bar=1.05, tfluid_k=293.15, name="Junction 3")    

# create junction elements
ext_grid = pp.create_ext_grid(net, junction=j1, p_bar=1.1, t_k=293.15, name="Grid Connection")
sink = pp.create_sink(net, junction=j3, mdot_kg_per_s=0.045, name="Sink")

# create branch element
pipe = pp.create_pipe_from_parameters(net, from_junction=j1, to_junction=j2, length_km=0.1, diameter_m=0.05, name="Pipe 1")
valve = pp.create_valve(net, from_junction=j2, to_junction=j3, diameter_m=0.05, opened=True, name="Valve 1")

Note that the fluid used here is lgas. You can find 5 predefined fluids in pandapipes:
    - lgas
    - hgas
    - hydrogen
    - water
    - air

And that the predefined valve element is an ideal valve. 

## Data Structure

Each dataframe in a pandapipes net object contains the information about one pandapipes element, such as pipe, sink, valve etc.

In [2]:
net.junction

,name,pn_bar,tfluid_k,height_m,in_service,type
0,Junction 1,1.05,293.15,0.0,True,junction
1,Junction 2,1.05,293.15,0.0,True,junction
2,Junction 3,1.05,293.15,0.0,True,junction


In [3]:
net.pipe

,name,from_junction,to_junction,std_type,length_km,diameter_m,k_mm,loss_coefficient,alpha_w_per_m2k,text_k,qext_w,sections,in_service,type
0,Pipe 1,0,1,None,0.1,0.05,1.0,0.0,0.0,293.0,0.0,1,True,pipe


In [4]:
net.ext_grid

,name,junction,p_bar,t_k,in_service,type
0,Grid Connection,0,1.1,293.15,True,pt


## Pipe Flow

We now run a pipe flow:


In [5]:
pp.pipeflow(net)

hp.pandapipes.pipeflow - INFO: niter 0
hp.pandapipes.pipeflow - INFO: niter 1
hp.pandapipes.pipeflow - INFO: niter 2
hp.pandapipes.pipeflow - INFO: niter 3
hp.pandapipes.pipeflow - INFO: ---------------------------------------------------------------------------------
hp.pandapipes.pipeflow - INFO: Calculation completed. Preparing results...
hp.pandapipes.pipeflow - INFO: Converged after 4 iterations.
hp.pandapipes.pipeflow - INFO: Norm of residual: 1.6351116571833302e-08
hp.pandapipes.pipeflow - INFO: tol_p: 0.0001
hp.pandapipes.pipeflow - INFO: tol_v: 0.0001


And check out at the results for junctions and pipes:

In [6]:
net.res_junction

,p_bar,t_k
0,1.100000,293.15
1,0.913916,293.15
2,0.913916,293.15


They're is no pressure loss between junction 2 and junction 3 because of the ideal valve.

In [7]:
net.res_pipe

,v_from_m_per_s,v_to_m_per_s,v_mean_m_per_s,p_from_bar,p_to_bar,t_from_k,t_to_k,mdot_from_kg_per_s,mdot_to_kg_per_s,vdot_norm_m3_per_s,reynolds,lambda,normfactor_from,normfactor_to
0,15.766074,17.295535,16.483877,1.1,0.913916,293.15,293.15,0.045,-0.045,0.06044,96710.314701,0.049222,0.512189,0.561877


### Closed valve

We now close the valve between junction 2 and junction 3:

In [8]:
# Problem can not run pipeflow if valve closed
net.valve.opened = False

The closed valve cuts the sink from the external grid:

<img src="pics/minimal_example_closed_valve.png">

This can be verified by running a power flow and inspecting the results. The pressure and temperature at junction 2 is given as NaN:

In [9]:
pp.pipeflow(net)
net.res_junction

hp.pandapipes.pipeflow_setup - INFO: Setting the following nodes out of service for hydraulics calculation in connectivity check:
In table junction: [2]
hp.pandapipes.pipeflow - INFO: niter 0
hp.pandapipes.pipeflow - INFO: niter 1
hp.pandapipes.pipeflow - INFO: ---------------------------------------------------------------------------------
hp.pandapipes.pipeflow - INFO: Calculation completed. Preparing results...
hp.pandapipes.pipeflow - INFO: Converged after 2 iterations.
hp.pandapipes.pipeflow - INFO: Norm of residual: 6.635561679383765e-07
hp.pandapipes.pipeflow - INFO: tol_p: 0.0001
hp.pandapipes.pipeflow - INFO: tol_v: 0.0001


,p_bar,t_k
0,1.1,293.15
1,1.1,293.15
2,NaN,NaN


Also the results from the pipe show that the mass flow is almost zero and the speed of the mass flow is zero. 

In [10]:
net.res_pipe

,v_from_m_per_s,v_to_m_per_s,v_mean_m_per_s,p_from_bar,p_to_bar,t_from_k,t_to_k,mdot_from_kg_per_s,mdot_to_kg_per_s,vdot_norm_m3_per_s,reynolds,lambda,normfactor_from,normfactor_to
0,0.0,0.0,0.0,1.1,1.1,293.15,293.15,0.0,-0.0,0.0,0.0,0.04856,0.512189,0.512189
